In [53]:
import keras


    

In [79]:
def rgb_model():
    image_w, image_h = 28, 28
    
    from keras.layers import Input, Dense, Dropout, Flatten
    from keras.layers import Conv2D, Activation, MaxPooling2D, BatchNormalization
    from keras.layers import GlobalMaxPooling1D
    
    kernels = 10
    kernel_size = (3,3)
    
    image_shape = (image_w, image_h, 3)
    
    model = keras.Sequential([
        
        Conv2D(kernels, kernel_size, use_bias=False, padding='valid', input_shape=image_shape),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2), strides=None, padding='valid'),
        
        Conv2D(kernels, kernel_size, use_bias=False, padding='valid', input_shape=image_shape),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2), strides=None, padding='valid'),
        
        Conv2D(kernels, kernel_size, use_bias=False, padding='same', input_shape=image_shape),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2), strides=None, padding='valid'),
        
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid'),
    ])
    
    return model
    
rgb_model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_84 (Conv2D)           (None, 26, 26, 10)        270       
_________________________________________________________________
batch_normalization_26 (Batc (None, 26, 26, 10)        40        
_________________________________________________________________
activation_83 (Activation)   (None, 26, 26, 10)        0         
_________________________________________________________________
max_pooling2d_81 (MaxPooling (None, 13, 13, 10)        0         
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 11, 11, 10)        900       
_________________________________________________________________
batch_normalization_27 (Batc (None, 11, 11, 10)        40        
_________________________________________________________________
activation_84 (Activation)   (None, 11, 11, 10)        0         
__________

In [76]:
def spec_model_multiinstance():
    spec_bands = 64
    frames = 10
    bag_size = 850
    kernels = 16
    kernel_size = (3,3)
    
    from keras.layers import Input, Dense, Dropout, Flatten
    from keras.layers import Conv2D, Activation, MaxPooling2D, BatchNormalization
    from keras.layers import GlobalMaxPooling1D

    frame_classifier = keras.Sequential([
        Conv2D(kernels, kernel_size, use_bias=True, padding='valid',
               strides=(1, 1), dilation_rate=(1, 1),
               input_shape=(frames, spec_bands, 1),
        ),
        Activation('relu'),
        MaxPooling2D((2, 2), strides=None, padding='valid'),
        
        Conv2D(kernels, kernel_size, use_bias=True, padding='valid',
               strides=(1, 1), dilation_rate=(1, 1),
        ),
        Activation('relu'),
        MaxPooling2D((2, 2), strides=None, padding='valid'),
        
        Flatten(),
        
        Dense(100, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid'),
    ])
    #frame_classifier.build()
    frame_classifier.summary()

    sample_shape = (100, frames, spec_bands, 1)
    sequence = Input(shape=sample_shape)
    wrap = keras.layers.TimeDistributed(frame_classifier, input_shape=sample_shape)(sequence)
    wrap = GlobalMaxPooling1D()(wrap)
    
    model = keras.Model(inputs=sequence, outputs=wrap)
    
    model.compile(optimizer='rmsprop', metrics=['acc'], loss='binary_crossentropy')
    
    return model
    
m = spec_model_multiinstance()
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_79 (Conv2D)           (None, 8, 62, 16)         160       
_________________________________________________________________
activation_78 (Activation)   (None, 8, 62, 16)         0         
_________________________________________________________________
max_pooling2d_76 (MaxPooling (None, 4, 31, 16)         0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 2, 29, 16)         2320      
_________________________________________________________________
activation_79 (Activation)   (None, 2, 29, 16)         0         
_________________________________________________________________
max_pooling2d_77 (MaxPooling (None, 1, 14, 16)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 224)               0         
__________